# Paso 1.   Webscraping

In [ ]:
import json
import requests
from bs4 import BeautifulSoup
import csv
import time
import os
import re


ruta_json = "# === NOTE: Replace with local path ==="


ruta_csv = "# === NOTE: Replace with local path ==="

def limpiar_para_url(texto):
    texto = texto.strip().upper()
    # Cambiar espacios por guiones
    texto = re.sub(r'\s+', '-', texto)
    # Quitar caracteres que no sean letras, números o guiones
    texto = re.sub(r'[^A-Z0-9\-]', '', texto)
    return texto

def construir_url(profesor):
    nombre_url = f"{limpiar_para_url(profesor['n'])}-{limpiar_para_url(profesor['a'])}_{profesor['i']}"
    url = f"https://www.misprofesores.com/profesores/{nombre_url}"
    return url

def extraer_reseñas(html):
    soup = BeautifulSoup(html, 'html.parser')
    tags = soup.find_all('span', class_='tag-box-choosetags')
    reseñas = []
    for tag in tags:
        texto = tag.get_text(strip=True)
        texto_limpio = texto.rsplit('(', 1)[0].strip()
        reseñas.append(texto_limpio)
    return reseñas

def main():
    # Leer JSON
    with open(ruta_json, "r", encoding="utf-8") as f:
        profesores = json.load(f)

    with open(ruta_csv, 'w', newline='', encoding='utf-8') as csvfile:
        campos = ['Nombre Completo', 'Departamento', 'Calificación', 'Comentarios']
        writer = csv.DictWriter(csvfile, fieldnames=campos)
        writer.writeheader()

        for prof in profesores:
            url = construir_url(prof)
            print(f"Obteniendo datos de: {url}")
            try:
                response = requests.get(url)
                response.raise_for_status()
                reseñas = extraer_reseñas(response.text)
                comentarios_texto = ", ".join(reseñas)
            except Exception as e:
                print(f"Error al obtener reseñas de {prof['n']} {prof['a']}: {e}")
                comentarios_texto = ""

            nombre_completo = f"{prof['n']} {prof['a']}"
            writer.writerow({
                'Nombre Completo': nombre_completo,
                'Departamento': prof['d'],
                'Calificación': prof['c'],
                'Comentarios': comentarios_texto
            })
            time.sleep(1)

    print(f"\nArchivo CSV generado en: {ruta_csv}")

if __name__ == "__main__":
    main()


# Paso 2. Procesamiento de pdfs

In [ ]:
import os
import pdfplumber
import pandas as pd
from pathlib import Path
import logging


def procesar_pdf_carpetas(carpeta_pdf, salida_csv):     
    """     
    Procesa todos los archivos PDF en una carpeta y extrae el texto línea por línea a un CSV.          
    Args:         
        carpeta_pdf (str): Ruta de la carpeta que contiene los PDFs         
        salida_csv (str): Ruta del archivo CSV de salida     
    """          
    logging.basicConfig(level=logging.INFO)     
    logger = logging.getLogger(__name__)          
    datos = []          
    try:                  
        if not os.path.exists(carpeta_pdf):             
            raise FileNotFoundError(f"La carpeta {carpeta_pdf} no existe")                           
        archivos_pdf = [f for f in os.listdir(carpeta_pdf) if f.lower().endswith('.pdf')]                  
        if not archivos_pdf:             
            logger.warning(f"No se encontraron archivos PDF en {carpeta_pdf}")             
            return                  
        logger.info(f"Encontrados {len(archivos_pdf)} archivos PDF")                  
        for archivo in archivos_pdf:             
            ruta_pdf = os.path.join(carpeta_pdf, archivo)             
            logger.info(f"Procesando: {archivo}")                          
            try:                 
                with pdfplumber.open(ruta_pdf) as pdf:                     
                    for num_pagina, pagina in enumerate(pdf.pages, 1):                         
                        try:                             
                            texto = pagina.extract_text()                             
                            if texto:                                 
                                lineas = texto.split('\n')                                 
                                for linea in lineas:                                     
                                    linea_limpia = linea.strip()                                     
                                    if linea_limpia:                                           
                                        datos.append({                                             
                                            'Archivo': archivo,                                             
                                            'Pagina': num_pagina,                                             
                                            'Linea': linea_limpia                                         
                                        })                         
                        except Exception as e:                             
                            logger.error(f"Error procesando página {num_pagina} de {archivo}: {e}")                             
                            continue                                              
                logger.info(f"✓ Completado: {archivo}")                              
            except Exception as e:                 
                logger.error(f"Error procesando {archivo}: {e}")                 
                continue                           
        if datos:             
            df = pd.DataFrame(datos)                                       
            Path(salida_csv).parent.mkdir(parents=True, exist_ok=True)                          
            df.to_csv(salida_csv, index=False, encoding='utf-8')             
            logger.info(f"✓ Guardado exitosamente en {salida_csv}")             
            logger.info(f"Total de líneas extraídas: {len(datos)}")         
        else:             
            logger.warning("No se extrajeron datos de ningún archivo")                  
    except Exception as e:         
        logger.error(f"Error general: {e}")         
        raise  

def procesar_pdf_individual(ruta_pdf, salida_csv):     
    """     
    Procesa un solo archivo PDF.          
    Args:         
        ruta_pdf (str): Ruta del archivo PDF         
        salida_csv (str): Ruta del archivo CSV de salida     
    """     
    carpeta_pdf = os.path.dirname(ruta_pdf)     
    archivo_pdf = os.path.basename(ruta_pdf)               
    procesar_pdf_carpetas(carpeta_pdf, salida_csv)   

if __name__ == "__main__":          
    carpeta_pdf = "# === NOTE: Replace with local path ==="     
    salida_csv = "# === NOTE: Replace with local path ==="               
    procesar_pdf_carpetas(carpeta_pdf, salida_csv)

# Paso 3. Agrupación de la información extraída

In [ ]:
import pandas as pd
import re
import numpy as np
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Configuración para mostrar más columnas y filas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

def limpiar_texto(texto):
    """Limpia y normaliza texto"""
    if pd.isna(texto):
        return ""
    texto = str(texto).strip()
    # Eliminar caracteres especiales al inicio y final
    texto = re.sub(r'^[•·\-\*\s]+', '', texto)
    texto = re.sub(r'[•·\-\*\s]+$', '', texto)
    return texto.strip()

def es_nombre_profesor(texto):
    """
    Identifica si un texto corresponde a un nombre de profesor
    usando múltiples patrones y heurísticas
    """
    if pd.isna(texto) or texto.strip() == "":
        return False
    
    texto = str(texto).strip()
    
    # Patrón principal: APELLIDO APELLIDO, NOMBRE o variaciones
    patron_nombre_formal = r'^[A-ZÁÉÍÓÚÑ][A-ZÁÉÍÓÚÑ\s]+,\s*[A-ZÁÉÍÓÚÑ][a-záéíóúñ\s]+$'
    
    # Patrones adicionales para nombres
    patrones_nombre = [
        r'^[A-Z][A-Z\s]+,\s*[A-Z][A-Z\s]+$',  # TODO MAYÚSCULAS
        r'^[A-ZÁÉÍÓÚÑ][a-záéíóúñA-ZÁÉÍÓÚÑ\s]+,\s*[A-ZÁÉÍÓÚÑ][a-záéíóúñA-ZÁÉÍÓÚÑ\s]+$',  # Mixto
        r'^[A-Z]{2,}\s+[A-Z]{2,},\s*[A-Z]{2,}',  # Apellidos cortos mayúsculas
    ]
    
    # Verificar patrones de nombre
    for patron in [patron_nombre_formal] + patrones_nombre:
        if re.match(patron, texto):
            return True
    
    # Heurísticas adicionales
    # Si contiene coma y palabras en mayúscula
    if ',' in texto and len([word for word in texto.split() if word.isupper() and len(word) > 2]) >= 2:
        return True
    
    # Si es todo mayúsculas y tiene formato de nombre
    if texto.isupper() and ',' in texto and len(texto.split()) >= 3:
        return True
    
    return False

def es_materia_o_codigo(texto):
    """Identifica si un texto corresponde a una materia o código"""
    if pd.isna(texto):
        return False
    
    texto = str(texto).strip()
    
    # Patrones para materias
    patrones_materia = [
        r'.*\([A-Z]\d+\)',  # Texto con código entre paréntesis
        r'^[A-Z]\d+$',      # Solo código
        r'Semestre\s+\d+',   # Semestre X
        r'^\d+\s*$',         # Solo números
        r'^[IVX]+\s*$',      # Números romanos
    ]
    
    for patron in patrones_materia:
        if re.match(patron, texto):
            return True
    
    return False

def es_contenido_irrelevante(texto):
    """Identifica contenido que no es reseña ni nombre"""
    if pd.isna(texto):
        return True
    
    texto = str(texto).strip()
    
    if texto == "" or texto == "•" or texto == "-":
        return True
    
    # Patrones de contenido irrelevante
    patrones_irrelevantes = [
        r'^\d+\s*$',  # Solo números
        r'^\.\.\.*$',  # Solo puntos
        r'^[•\-\*\s]*$',  # Solo símbolos de lista
        r'Semestre\s+\d+',
        r'^\d+\s*\.\s*$',  # Numeración
        r'^Semestre',
        r'^\s*$'  # Solo espacios
    ]
    
    for patron in patrones_irrelevantes:
        if re.match(patron, texto):
            return True
    
    return False

def es_resena_valida(texto):
    """Valida si un texto es una reseña válida"""
    if pd.isna(texto) or texto.strip() == "":
        return False
    
    texto = str(texto).strip()
    
    # Eliminar símbolos de lista
    texto_limpio = re.sub(r'^[•·\-\*\s]+', '', texto)
    
    if len(texto_limpio) < 3:  # Muy corto
        return False
    
    if es_nombre_profesor(texto) or es_materia_o_codigo(texto) or es_contenido_irrelevante(texto):
        return False
    
    # Debe tener contenido significativo
    palabras = texto_limpio.split()
    if len(palabras) < 2:
        return False
    
    return True

def procesar_csv_resenas(filepath):
    """Procesa el CSV y agrupa reseñas por profesor"""
    
    print("Cargando archivo CSV...")
    try:
        df = pd.read_csv(filepath, encoding='utf-8')
    except UnicodeDecodeError:
        try:
            df = pd.read_csv(filepath, encoding='latin-1')
        except:
            df = pd.read_csv(filepath, encoding='cp1252')
    
    print(f"Archivo cargado. Total de filas: {len(df)}")
    print(f"Columnas: {list(df.columns)}")
    
    # Renombrar columnas para consistencia
    if len(df.columns) >= 3:
        df.columns = ['Archivo', 'Pagina', 'Texto']
    
    # Limpiar texto
    df['Texto_Limpio'] = df['Texto'].apply(limpiar_texto)
    
    # Filtrar filas vacías
    df = df[df['Texto_Limpio'] != ""]
    
    print(f"Después de limpiar: {len(df)} filas")
    
    # Crear estructura para almacenar resultados
    resultados = []
    
    # Variables de estado
    profesor_actual = None
    materia_actual = None
    resenas_actuales = []
    
    print("Procesando filas...")
    
    for idx, row in df.iterrows():
        texto = row['Texto_Limpio']
        archivo = row['Archivo']
        pagina = row['Pagina']
        
        if idx % 1000 == 0:
            print(f"Procesando fila {idx}/{len(df)}...")
        
        # Verificar tipo de contenido
        if es_materia_o_codigo(texto):
            # Guardar reseñas anteriores si existen
            if profesor_actual and resenas_actuales:
                resena_completa = ' '.join(resenas_actuales)
                resultados.append({
                    'Archivo': archivo,
                    'Pagina': pagina,
                    'Profesor': profesor_actual,
                    'Materia': materia_actual,
                    'Resena': resena_completa,
                    'Num_Fragmentos': len(resenas_actuales)
                })
            
            # Nueva materia
            materia_actual = texto
            profesor_actual = None
            resenas_actuales = []
            
        elif es_nombre_profesor(texto):
            # Guardar reseñas del profesor anterior
            if profesor_actual and resenas_actuales:
                resena_completa = ' '.join(resenas_actuales)
                resultados.append({
                    'Archivo': archivo,
                    'Pagina': pagina,
                    'Profesor': profesor_actual,
                    'Materia': materia_actual,
                    'Resena': resena_completa,
                    'Num_Fragmentos': len(resenas_actuales)
                })
            
            # Nuevo profesor
            profesor_actual = texto
            resenas_actuales = []
            
        elif es_resena_valida(texto):
            # Agregar reseña al profesor actual
            if profesor_actual:
                # Limpiar símbolos de lista
                texto_resena = re.sub(r'^[•·\-\*\s]+', '', texto)
                if texto_resena:
                    resenas_actuales.append(texto_resena)
        
        elif es_contenido_irrelevante(texto):
            # Ignorar contenido irrelevante
            continue
    
    # Procesar última reseña
    if profesor_actual and resenas_actuales:
        resena_completa = ' '.join(resenas_actuales)
        resultados.append({
            'Archivo': df.iloc[-1]['Archivo'],
            'Pagina': df.iloc[-1]['Pagina'],
            'Profesor': profesor_actual,
            'Materia': materia_actual,
            'Resena': resena_completa,
            'Num_Fragmentos': len(resenas_actuales)
        })
    
    # Crear DataFrame con resultados
    df_resultados = pd.DataFrame(resultados)
    
    print(f"\nProcesamiento completado!")
    print(f"Total de reseñas agrupadas: {len(df_resultados)}")
    
    return df_resultados, df

def analizar_resultados(df_resultados):
    """Analiza los resultados obtenidos"""
    print("\n=== ANÁLISIS DE RESULTADOS ===")
    print(f"Total de reseñas procesadas: {len(df_resultados)}")
    
    if len(df_resultados) > 0:
        print(f"Profesores únicos: {df_resultados['Profesor'].nunique()}")
        print(f"Materias únicas: {df_resultados['Materia'].nunique()}")
        print(f"Archivos procesados: {df_resultados['Archivo'].nunique()}")
        
        # Estadísticas de fragmentos
        fragmentos_stats = df_resultados['Num_Fragmentos'].describe()
        print(f"\nEstadísticas de fragmentos por reseña:")
        print(fragmentos_stats)
        
        # Top profesores con más reseñas
        print(f"\nTop 10 profesores con más reseñas:")
        top_profesores = df_resultados['Profesor'].value_counts().head(10)
        print(top_profesores)
        
        # Ejemplos de reseñas
        print(f"\n=== EJEMPLOS DE RESEÑAS AGRUPADAS ===")
        for idx, row in df_resultados.head(5).iterrows():
            print(f"\nProfesor: {row['Profesor']}")
            print(f"Materia: {row['Materia']}")
            print(f"Reseña ({row['Num_Fragmentos']} fragmentos): {row['Resena'][:200]}...")
    
    return df_resultados

def guardar_resultados(df_resultados, output_path=None):
    """Guarda los resultados en CSV"""
    if output_path is None:
        output_path = r"# === NOTE: Replace with local path ==="
    
    df_resultados.to_csv(output_path, index=False, encoding='utf-8')
    print(f"\nResultados guardados en: {output_path}")
    
    return output_path

# Función principal
def main():
    # Ruta del archivo
    filepath = r"# === NOTE: Replace with local path ==="
    
    try:
        # Procesar archivo
        df_resultados, df_original = procesar_csv_resenas(filepath)
        
        # Analizar resultados
        df_resultados = analizar_resultados(df_resultados)
        
        # Guardar resultados
        output_path = guardar_resultados(df_resultados)
        
        print(f"\n✅ Proceso completado exitosamente!")
        print(f"📁 Archivo original: {len(df_original)} filas")
        print(f"📊 Reseñas agrupadas: {len(df_resultados)} reseñas")
        print(f"💾 Guardado en: {output_path}")
        
        return df_resultados, df_original
        
    except Exception as e:
        print(f"❌ Error durante el procesamiento: {str(e)}")
        import traceback
        traceback.print_exc()
        return None, None


if __name__ == "__main__":
    df_agrupado, df_raw = main()

# Paso 4. Unificación de datasets

In [ ]:
import pandas as pd
import numpy as np
import re
from fuzzywuzzy import fuzz, process
import unicodedata

class UnificadorDatasetsProfesores:
    def __init__(self):
        self.threshold_nombres = 85  # Umbral para matching de nombres
        
    def limpiar_nombre(self, nombre):
        """Limpia y normaliza nombres"""
        if pd.isna(nombre):
            return ""
        
        nombre = str(nombre).upper().strip()
        # Remover acentos
        nombre = unicodedata.normalize('NFD', nombre)
        nombre = ''.join(c for c in nombre if unicodedata.category(c) != 'Mn')
        
        return nombre
    
    def extraer_nombres_apellidos(self, nombre):
        """Extrae nombres y apellidos en formato estándar"""
        nombre_limpio = self.limpiar_nombre(nombre)
        
        # Patrón: "APELLIDO APELLIDO, NOMBRE NOMBRE"
        if ',' in nombre_limpio:
            partes = nombre_limpio.split(',', 1)
            apellidos = partes[0].strip()
            nombres = partes[1].strip()
        else:
            # Sin coma, separar palabras
            palabras = nombre_limpio.split()
            if len(palabras) == 1:
                apellidos = palabras[0]
                nombres = ""
            elif len(palabras) == 2:
                nombres = palabras[0]
                apellidos = palabras[1]
            else:
                # Asumir últimas 2 palabras como apellidos
                apellidos = ' '.join(palabras[-2:])
                nombres = ' '.join(palabras[:-2])
        
        return apellidos.strip(), nombres.strip()
    
    def crear_variaciones_nombre(self, nombre):
        """Crea variaciones posibles de un nombre"""
        apellidos, nombres = self.extraer_nombres_apellidos(nombre)
        
        variaciones = [
            f"{apellidos}, {nombres}",  # Formato completo
            f"{apellidos}",  # Solo apellidos
            f"{nombres} {apellidos}",  # Orden normal
            apellidos.split()[0] if apellidos else "",  # Primer apellido
        ]
        
        return [v for v in variaciones if v.strip()]
    
    def encontrar_mejor_match(self, nombre_buscar, lista_nombres):
        """Encuentra el mejor match usando fuzzy matching"""
        variaciones = self.crear_variaciones_nombre(nombre_buscar)
        mejor_match = None
        mejor_score = 0
        
        for variacion in variaciones:
            if not variacion:
                continue
                
            for nombre_candidato in lista_nombres:
                variaciones_candidato = self.crear_variaciones_nombre(nombre_candidato)
                
                for var_candidato in variaciones_candidato:
                    score = fuzz.ratio(variacion, var_candidato)
                    if score > mejor_score:
                        mejor_score = score
                        mejor_match = nombre_candidato
        
        return mejor_match if mejor_score >= self.threshold_nombres else None, mejor_score
    
    def cargar_datasets(self, ruta_dataset1, ruta_dataset2):
        """Carga ambos datasets"""
        
        df1 = pd.read_csv(ruta_dataset1)
        print(f"Dataset 1 cargado: {len(df1)} filas")
        print("Columnas:", df1.columns.tolist())
        
         
        df2 = pd.read_csv(ruta_dataset2)
        print(f"Dataset 2 cargado: {len(df2)} filas")
        print("Columnas:", df2.columns.tolist())
        
        return df1, df2
    
    def unificar_datasets(self, ruta_dataset1, ruta_dataset2):
        """Función principal que unifica ambos datasets"""
        
        # Cargar datasets
        df1, df2 = self.cargar_datasets(ruta_dataset1, ruta_dataset2)
        
        # Preparar dataset 1 
        df1_procesado = df1.copy()
        df1_procesado['nombre_normalizado'] = df1_procesado['Nombre Completo'].apply(self.limpiar_nombre)
        df1_procesado['fuente'] = 'dataset1'
        
        # Preparar dataset 2 
        df2_procesado = df2.copy()
        df2_procesado['nombre_normalizado'] = df2_procesado['Profesor'].apply(self.limpiar_nombre)
        df2_procesado['fuente'] = 'dataset2'
        
        # Crear estructura unificada
        dataset_unificado = []
        matches_encontrados = {}
        
        # Procesar dataset 1
        print("\nProcesando Dataset 1...")
        for idx, row in df1_procesado.iterrows():
            registro = {
                'id': f"d1_{idx}",
                'nombre_original': row['Nombre Completo'],
                'nombre_normalizado': row['nombre_normalizado'],
                'departamento': row.get('Departamento', ''),
                'materia': '',  # No tiene materia específica
                'calificacion_numerica': row.get('Calificación', None),
                'comentarios': row.get('Comentarios', ''),
                'resena_detallada': '',
                'archivo_fuente': '',
                'pagina': None,
                'num_fragmentos': None,
                'fuente': 'dataset1',
                'match_id': None
            }
            dataset_unificado.append(registro)
        
        # Procesar dataset 2 y buscar matches
        print("Procesando Dataset 2 y buscando matches...")
        nombres_d1 = df1_procesado['Nombre Completo'].tolist()
        
        for idx, row in df2_procesado.iterrows():
            # Buscar match en dataset 1
            mejor_match, score = self.encontrar_mejor_match(
                row['Profesor'], nombres_d1
            )
            
            registro = {
                'id': f"d2_{idx}",
                'nombre_original': row['Profesor'],
                'nombre_normalizado': row['nombre_normalizado'],
                'departamento': '',  # Se puede inferir de la materia
                'materia': row.get('Materia', ''),
                'calificacion_numerica': None,
                'comentarios': '',
                'resena_detallada': row.get('Resena', ''),
                'archivo_fuente': row.get('Archivo', ''),
                'pagina': row.get('Pagina', None),
                'num_fragmentos': row.get('Num_Fragmentos', None),
                'fuente': 'dataset2',
                'match_id': mejor_match if score >= self.threshold_nombres else None,
                'match_score': score
            }
            
            if mejor_match:
                if mejor_match not in matches_encontrados:
                    matches_encontrados[mejor_match] = []
                matches_encontrados[mejor_match].append(registro['id'])
                print(f"Match encontrado: {row['Profesor']} -> {mejor_match} (Score: {score})")
            
            dataset_unificado.append(registro)
        
        # Convertir a DataFrame
        df_unificado = pd.DataFrame(dataset_unificado)
        
        # Estadísticas
        print(f"\n=== ESTADÍSTICAS DE UNIFICACIÓN ===")
        print(f"Total registros unificados: {len(df_unificado)}")
        print(f"Del Dataset 1: {len(df_unificado[df_unificado['fuente'] == 'dataset1'])}")
        print(f"Del Dataset 2: {len(df_unificado[df_unificado['fuente'] == 'dataset2'])}")
        print(f"Matches encontrados: {len(matches_encontrados)}")
        print(f"Profesores únicos (aproximado): {df_unificado['nombre_normalizado'].nunique()}")
        
        return df_unificado, matches_encontrados
    
    def consolidar_por_profesor(self, df_unificado):
        """Consolida registros del mismo profesor"""
        
        profesores_consolidados = []
        
        # Agrupar por nombre normalizado
        grupos = df_unificado.groupby('nombre_normalizado')
        
        for nombre_norm, grupo in grupos:
            # Tomar el primer registro como base
            registro_base = grupo.iloc[0].copy()
            
            # Combinar información de ambos datasets
            comentarios_d1 = []
            resenas_d2 = []
            materias = []
            calificaciones = []
            archivos = []
            
            for _, row in grupo.iterrows():
                if row['fuente'] == 'dataset1':
                    if row['comentarios']:
                        comentarios_d1.append(row['comentarios'])
                    if pd.notna(row['calificacion_numerica']):
                        calificaciones.append(row['calificacion_numerica'])
                        
                elif row['fuente'] == 'dataset2':
                    if row['resena_detallada']:
                        resenas_d2.append(row['resena_detallada'])
                    if row['materia']:
                        materias.append(row['materia'])
                    if row['archivo_fuente']:
                        archivos.append(row['archivo_fuente'])
            
            # Crear registro consolidado
            profesor_consolidado = {
                'nombre': registro_base['nombre_original'],
                'nombre_normalizado': nombre_norm,
                'departamento': registro_base['departamento'],
                'materias': list(set(materias)),  # Materias únicas
                'calificacion_promedio': np.mean(calificaciones) if calificaciones else None,
                'num_calificaciones': len(calificaciones),
                'comentarios_estructurados': comentarios_d1,
                'resenas_detalladas': resenas_d2,
                'total_comentarios': len(comentarios_d1) + len(resenas_d2),
                'archivos_fuente': list(set(archivos)),
                'datasets_presente': list(grupo['fuente'].unique())
            }
            
            profesores_consolidados.append(profesor_consolidado)
        
        return pd.DataFrame(profesores_consolidados)

# Ejemplo de uso
if __name__ == "__main__":
    
    unificador = UnificadorDatasetsProfesores()
    
    # Rutas de tus archivos
    ruta_dataset1 = r"# === NOTE: Replace with local path ==="
    ruta_dataset2 = r"# === NOTE: Replace with local path ==="
    
    # Unificar datasets
    df_unificado, matches = unificador.unificar_datasets(ruta_dataset1, ruta_dataset2)
    
    # Guardar resultado intermedio
    df_unificado.to_csv("dataset_unificado_raw.csv", index=False)
    print("\nDataset unificado guardado como 'dataset_unificado_raw.csv'")
    
    # Consolidar por profesor
    df_consolidado = unificador.consolidar_por_profesor(df_unificado)
    
    # Guardar resultado final
    df_consolidado.to_csv("profesores_consolidados.csv", index=False)
    print("Dataset consolidado guardado como 'profesores_consolidados.csv'")
    
    # Mostrar algunos ejemplos
    print("\n=== EJEMPLOS DE PROFESORES CONSOLIDADOS ===")
    for idx, profesor in df_consolidado.head(3).iterrows():
        print(f"\nProfesor: {profesor['nombre']}")
        print(f"Materias: {profesor['materias']}")
        print(f"Calificación promedio: {profesor['calificacion_promedio']}")
        print(f"Total comentarios: {profesor['total_comentarios']}")
        print(f"Presente en datasets: {profesor['datasets_presente']}")